# **downloading and wrangling skin fibroblast mutations from somaMutDB**

- qebsite download from https://vijglab.einsteinmed.org/SomaMutDB

### hg19_101
- Accurate identification of single nucleotide variants in whole genome amplified single cells

### hg19_7
- UV-exposure, endogenous DNA damage, and DNA replication errors shape the spectra of genome changes in human skin
- what do the D1-R-H1 files vs DAG files mean??????????????

In [16]:
%%bash 

wget http://vijglab.einsteinmed.org/static/vcf/hg19_7.tar.gz --no-check-certificate -P ../../../data/skin/mutations/ -o  ../../../data/skin/mutations/wget_7_output.txt
    
# mkdir ../../../data/skin/mutations/SomaMutDB/hg19_7/

tar -xzf ../../../data/skin/mutations/hg19_7.tar.gz -C ../../../data/skin/mutations/SomaMutDB/hg19_7/
#https://askubuntu.com/questions/25347/what-command-do-i-need-to-unzip-extract-a-tar-gz-file
#https://askubuntu.com/questions/45349/how-to-extract-files-to-another-directory-using-tar-command

#create a file with all the list names 
cd ../../../data/skin/mutations/SomaMutDB/
ls hg19_7/ > list_7_muts.txt

#grep each file in the data file list above into one combined. 

while read p; do
  grep -v "^#" hg19_7/$p >>allMuts_7.txt
done <list_7_muts.txt
#and check the # mutant lines 
wc -l allMuts_7.txt

15270 allMuts_7.txt


In [18]:
%%bash
wget http://vijglab.einsteinmed.org/static/vcf/hg19_101.tar.gz --no-check-certificate -P ../../../data/skin/mutations/ -o  ../../../data/skin/mutations/wget_101_output.txt
    
# mkdir ../../../data/skin/mutations/SomaMutDB/hg19_101/
    
tar -xzf ../../../data/skin/mutations/hg19_101.tar.gz -C ../../../data/skin/mutations/SomaMutDB/hg19_101/
    
#create a file with the file snames to loop through 
cd ../../../data/skin/mutations/SomaMutDB/
ls hg19_101/*SCcaller.vcf > list_101_muts.txt  

#loop trhough reading in the files and combine into one 
while read p; do
  grep -v "^#" $p >>allMuts_101.txt
done <list_101_muts.txt
#check the final mut line count 
wc -l allMuts_101.txt

10278 allMuts_101.txt


## **combine and process**

In [19]:
%%bash
cd ../../../data/skin/mutations/SomaMutDB/
cat allMuts_7.txt > allMuts.txt
cat allMuts_101.txt >> allMuts.txt

In [20]:
%%bash
cd ../../../data/skin/mutations/SomaMutDB/
awk '{print "chr"$1"\t"$2"\t"$2+1"\t"$4"|"$5"|"$3"|"$6"|"$7"|"$8"|"$9"|"$10}' allMuts.txt > all_skin_muts_readyForLiftover_hg19.bed

In [21]:
%%bash 
cd ../../../data/skin/mutations/
../../../data/applications/UCSC_genomeBrowser_Blat/liftOver \
    SomaMutDB/all_skin_muts_readyForLiftover_hg19.bed \
    hg19ToHg18.over.chain.gz \
    SomaMutDB/skin_mutations_hg18.bed \
    SomaMutDB/liftover_unlifted_hg19_to_18.bed


Reading liftover chains
Mapping coordinates


In [22]:
!wc -l ../../../data/skin/mutations/SomaMutDB/liftover_unlifted_hg19_to_18.bed

84 ../../../data/skin/mutations/SomaMutDB/liftover_unlifted_hg19_to_18.bed


In [23]:
!wc -l ../../../data/skin/mutations/SomaMutDB/skin_mutations_hg18.bed

25506 ../../../data/skin/mutations/SomaMutDB/skin_mutations_hg18.bed


In [24]:
%%bash 
cd ../../../data/skin/mutations/SomaMutDB/
sort -k1,1 -k2,2n skin_mutations_hg18.bed > skin_mutations_hg18_sorted.bed

In [25]:
%%bash 
cd ../../../data/skin/mutations/SomaMutDB/
sed 's/|/\t/g' skin_mutations_hg18_sorted.bed > skin_mutations.bed #https://stackoverflow.com/questions/525592/find-and-replace-inside-a-text-file-from-a-bash-command


In [26]:
%%bash 
cd ../../../data/skin/mutations/SomaMutDB/
head skin_mutations.bed #https://stackoverflow.com/questions/525592/find-and-replace-inside-a-text-file-from-a-bash-command


chr1	744003	744004	T	C	.	.	.	.	.	.
chr1	744003	744004	T	C	.	.	.	.	.	.
chr1	744003	744004	T	C	.	.	.	.	.	.
chr1	744003	744004	T	C	.	.	.	.	.	.
chr1	744003	744004	T	C	.	.	.	.	.	.
chr1	744003	744004	T	C	.	.	.	.	.	.
chr1	746317	746318	G	C	.	12.0	.	NS=1	GT:SO:AD:BI:GQ:PL	0/1:True:15,9:0.796:12:131,89,77,387
chr1	746317	746318	G	C	.	12.0	.	NS=1	GT:SO:AD:BI:GQ:PL	0/1:True:15,9:0.796:12:131,89,77,387
chr1	748371	748372	A	C	.	12.0	.	NS=1	GT:SO:AD:BI:GQ:PL	0/1:True:17,9:0.794:12:134,91,79,505
chr1	748371	748372	A	C	.	12.0	.	NS=1	GT:SO:AD:BI:GQ:PL	0/1:True:17,9:0.794:12:134,91,79,505


In [27]:
import pandas as pd 

In [28]:
test = pd.read_table("../../../data/skin/mutations/SomaMutDB/skin_mutations.bed",header=None)

In [29]:
test.columns = ["chrom","start","fake_end","ref","alt"]+list(test.columns)[5:]

In [142]:
! tail ../../../data/skin/mutations/SomaMutDB/allMuts_101.txt 

21	43114602	.	T	C	11.0	.	NS=1	GT:SO:AD:BI:GQ:PL	0/1:True:21,10:0.796:11:143,101,90,710
21	44111759	.	T	C	21.0	.	NS=1	GT:SO:AD:BI:GQ:PL	0/1:True:17,10:0.658:21:146,99,78,619
21	44156055	.	G	A	19.0	.	NS=1	GT:SO:AD:BI:GQ:PL	0/1:True:20,10:0.659:19:155,102,83,688
22	21091073	.	T	G	13.0	.	NS=1	GT:SO:AD:BI:GQ:PL	0/1:True:17,8:0.682:13:118,82,69,574
22	23486021	.	T	C	9.0	.	NS=1	GT:SO:AD:BI:GQ:PL	0/1:True:14,6:0.656:9:93,62,54,491
22	29618263	.	T	C	10.0	.	NS=1	GT:SO:AD:BI:GQ:PL	0/1:True:21,8:0.664:10:124,85,75,745
22	46497313	.	C	A	28.0	.	NS=1	GT:SO:AD:BI:GQ:PL	0/1:True:17,13:0.75:28:199,127,100,599
22	48051797	.	G	C	19.0	.	NS=1	GT:SO:AD:BI:GQ:PL	0/1:True:18,10:0.619:19:145,100,80,602
22	48051845	.	A	C	9.0	.	NS=1	GT:SO:AD:BI:GQ:PL	0/1:True:16,7:0.619:9:103,72,64,494
22	48051995	.	A	C	10.0	.	NS=1	GT:SO:AD:BI:GQ:PL	0/1:True:21,9:0.619:10:130,92,83,671


In [30]:
! awk '{print "chr"$1"\t"$2"\t"$2+1}' ../../../data/skin/mutations/SomaMutDB/allMuts_101.txt | sort -k1,1 -k2,2n > ../../../data/skin/mutations/SomaMutDB/allMuts_101_tabixTest.bed


In [32]:
!bgzip ../../../data/skin/mutations/SomaMutDB/allMuts_101_tabixTest.bed

In [1]:
!tabix -s 1 -b 2  -e 3 ../../../data/skin/mutations/SomaMutDB/allMuts_101_tabixTest.bed.gz

In [2]:
!tabix ../../../data/skin/mutations/SomaMutDB/allMuts_101_tabixTest.bed.gz chr21:43114370-43114371

chr21	43114370	43114371
chr21	43114370	43114371


# **testing the 0/1 based** ~~~~~~~~~~~~~~~~~~

In [1]:
import gzip 
from Bio import SeqIO

In [3]:
%%bash 
#going to test the "ref" col from the mutation dataset matches the base returned by the seq query 
grep "chr3" ../../../data/skin/mutations/SomaMutDB/skin_mutations_hg18.bed > ../../../data/skin/mutations/SomaMutDB/test_chr3_muts_hg18.bed

In [35]:
# chr3_muts_lines = open("../../../data/skin/mutations/SomaMutDB/test_chr3_muts_hg18.bed").readlines()

# for line in chr3_muts_lines: 
#     assert line.split("\t")[0] == "chr3"
all_muts_readlines = open("../../../data/skin/mutations/SomaMutDB/skin_mutations_hg18.bed").readlines()
all_muts_readlines.sort()

misaligned_lines = []
aligned_lines = []
indel_lines=[]
count_n = 0
for i in range(1,23): 
    cur_chrom = "chr"+str(i)
    with gzip.open("../../../data/global/sequence/chr{}.fa.gz".format(i), "rt") as handle: #https://stackoverflow.com/questions/42757283/seqio-parse-on-a-fasta-gz
        for record in SeqIO.parse(handle,"fasta"): 
            for line in all_muts_readlines: 
                if line.split()[0] == cur_chrom: 
                    mut_pos = int(line.split("\t")[1])-1
                    mut_data_ref = line.split("\t")[3].rstrip("\n")
                    mut_data_alt = line.split("\t")[4].rstrip("\n")
                    hg18_seq_base = record.seq[mut_pos].upper()
        #             if hg18_seq_base != "N":
        #                 assert mut_data_ref == hg18_seq_base
                    if len(mut_data_ref)==1: 
                        if mut_data_ref != hg18_seq_base: 
                            misaligned_lines.append([mut_pos, mut_data_ref, hg18_seq_base, str(record.seq[mut_pos-2:mut_pos+3]).upper(),line])
                        else: 
                            aligned_lines.append([mut_pos, mut_data_ref, hg18_seq_base, str(record.seq[mut_pos-2:mut_pos+3]).upper(),line])
                    else: 
                        indel_lines.append([mut_pos, mut_data_ref, hg18_seq_base, str(record.seq[mut_pos-2:mut_pos+3]).upper(),line])


In [7]:
!wc -l ../../../data/skin/mutations/SomaMutDB/skin_mutations_hg18.bed

2543 ../../../data/skin/mutations/SomaMutDB/skin_mutations_hg18.bed


In [36]:
print(len(misaligned_lines),"sites aren't aligned in chr3 out of",len(all_muts_readlines),". Error proportion is",len(misaligned_lines)/len(all_muts_readlines))

1 sites aren't aligned in chr3 out of 2543 . Error proportion is 0.00039323633503735744


In [43]:
len(aligned_lines), "total aligned SNPs avaulable for use"

(1546, 'total aligned SNPs avaulable for use')

In [44]:
 len(all_muts_readlines)-(len(indel_lines)+ len(aligned_lines)), "muts lost to sex chroms"

(105, 'muts lost to sex chroms')

In [65]:
print(len(misaligned_lines),"sites aren't aligned in chr3 out of",len(chr3_muts_lines),". Error proportion is",len(misaligned_lines)/len(chr3_muts_lines))

74 sites aren't aligned in chr3 out of 37706 . Error proportion is 0.001962552378931735


 ## **recoding to make sure its not my fault**

In [14]:
%%bash 
#download the mutations from UV-exposure, endogenous DNA damage, and DNA replication errors 
#shape the spectra of genome changes in human skin, hg19 version 
cd ../../../data/skin/mutations/SomaMutDB/
wget http://vijglab.einsteinmed.org/static/vcf/hg19_7.tar.gz \
    --no-check-certificate  -o  wget_7_output.txt
    
mkdir hg19_7/

tar -xzf hg19_7.tar.gz -C hg19_7/

In [16]:
%%bash 
cd ../../../data/skin/mutations/SomaMutDB/
ls hg19_7/* > hg19_7_filelist.txt

In [20]:
%%bash 
cd ../../../data/skin/mutations/SomaMutDB/

while read p; do
  grep -v "^#" $p >>allMuts_7.txt
done < hg19_7_filelist.txt

In [28]:
%%bash
cd ../../../data/skin/mutations/SomaMutDB/
awk '{print "chr"$1"\t"$2"\t"$2+1"\t"$4"\t"$5}' allMuts_7.txt > processedMuts_hg_7.txt

In [29]:
!head ../../../data/skin/mutations/SomaMutDB/processedMuts_hg_7.txt

chr1	107123428	107123429	AG	A
chr1	158074427	158074428	TG	T
chr2	6359430	6359431	ACT	A
chr2	50900803	50900804	TA	T
chr2	116275188	116275189	TC	T
chr2	171240283	171240284	G	A
chr2	216574848	216574849	TAAAGACCATTCTC	T
chr3	138724787	138724788	G	A
chr4	25430034	25430035	GT	G
chr4	36132113	36132114	TA	T


In [30]:
%%bash 
cd ../../../data/skin/mutations/
../../../data/applications/UCSC_genomeBrowser_Blat/liftOver \
    SomaMutDB/processedMuts_hg_7.txt \
    hg19ToHg18.over.chain.gz \
    SomaMutDB/skin_mutations_hg18.bed \
    SomaMutDB/liftover_unlifted_hg19_to_18.bed

Reading liftover chains
Mapping coordinates


In [31]:
!head ../../../data/skin/mutations/SomaMutDB/liftover_unlifted_hg19_to_18.bed

#Deleted in new
chr7	421448	421449	T	G
#Deleted in new
chr3	194039332	194039333	TG	T


In [32]:
%%bash 
head ../../../data/skin/mutations/SomaMutDB/skin_mutations_hg18.bed

chr1	106924951	106924952	AG	A
chr1	156341051	156341052	TG	T
chr2	6276881	6276882	ACT	A
chr2	50754307	50754308	TA	T
chr2	115991658	115991659	TC	T
chr2	170948529	170948530	G	A
chr2	216283093	216283094	TAAAGACCATTCTC	T
chr3	140207477	140207478	G	A
chr4	25039132	25039133	GT	G
chr4	35808508	35808509	TA	T


In [33]:
%%bash 
cd ../../../data/skin/mutations/SomaMutDB/
sort -k1,1 -k2,2n skin_mutations_hg18.bed -o skin_mutations_hg18.bed
bgzip skin_mutations_hg18.bed

In [38]:
%%bash 
cd ../../../data/skin/mutations/SomaMutDB/
bgzip -d skin_mutations_hg18.bed.gz

In [36]:
# %%bash 
# cd ../../../data/skin/mutations/SomaMutDB/
# tabix -b 1 -s 2 -e 3 skin_mutations_hg18.bed.gz

[E::get_intv] Failed to parse TBX_GENERIC, was wrong -p [type] used?
The offending line was: "chr1	744003	744004	T	C"
[E::get_intv] Failed to parse TBX_GENERIC, was wrong -p [type] used?
The offending line was: "chr1	1217822	1217823	TCCCCAGGGCAGCATTGCATAGTGACCAGCCGACTGGCCTTAG	T"
[E::get_intv] Failed to parse TBX_GENERIC, was wrong -p [type] used?
The offending line was: "chr1	1322733	1322734	G	GG"
[E::get_intv] Failed to parse TBX_GENERIC, was wrong -p [type] used?
The offending line was: "chr1	1336768	1336769	TAAGAA	T"
[E::get_intv] Failed to parse TBX_GENERIC, was wrong -p [type] used?
The offending line was: "chr1	1362609	1362610	C	T"
[E::get_intv] Failed to parse TBX_GENERIC, was wrong -p [type] used?
The offending line was: "chr1	1362610	1362611	C	T"
[E::get_intv] Failed to parse TBX_GENERIC, was wrong -p [type] used?
The offending line was: "chr1	1847543	1847544	G	A"
[E::get_intv] Failed to parse TBX_GENERIC, was wrong -p [type] used?
The offending line was: "chr1	1847544	1847545

## **do the comparison**

In [41]:
import pandas as pd 
import numpy as np
import gzip 
from Bio import SeqIO

In [52]:
%%bash 
cd ../../../data/skin/mutations/SomaMutDB/
grep "chr4" skin_mutations_hg18.bed > skin_mutations_hg18_chr4.bed

In [53]:
muts_readlines = open("../../../data/skin/mutations/SomaMutDB/skin_mutations_hg18_chr4.bed").readlines()

In [56]:
for line in muts_readlines: 
    assert line.split()[0] == "chr4"

In [60]:
muts_readlines[0:10]

['chr4\t1183778\t1183779\tC\tT\n',
 'chr4\t1567279\t1567280\tAC\tA\n',
 'chr4\t2014186\t2014187\tC\tCC\n',
 'chr4\t6425193\t6425194\tG\tA\n',
 'chr4\t9688565\t9688566\tG\tA\n',
 'chr4\t9688566\t9688567\tG\tA\n',
 'chr4\t10156296\t10156297\tT\tTT\n',
 'chr4\t11272057\t11272058\tA\tAA\n',
 'chr4\t11669945\t11669946\tAT\tA\n',
 'chr4\t12215529\t12215530\tAAGTATTTT\tA\n']

In [66]:
with gzip.open("../../../data/global/sequence/chr4.fa.gz", "rt") as handle: #https://stackoverflow.com/questions/42757283/seqio-parse-on-a-fasta-gz
    for record in SeqIO.parse(handle,"fasta"): 
        for line in  muts_readlines: 
            chrom,mut_pos,mut_pos_plus1,mut_data_ref,mut_data_alt = line.split("\t")
            mut_pos=int(mut_pos)
            hg18_seq_base = record.seq[mut_pos].upper()
#             if hg18_seq_base != "N":
#                 assert mut_data_ref == hg18_seq_base
            if 
            if mut_data_ref != hg18_seq_base: 
                misaligned_lines.append([mut_pos, mut_data_ref, hg18_seq_base, str(record.seq[mut_pos-2:mut_pos+3]).upper(),line])
            else: 
                aligned_lines.append([mut_pos, mut_data_ref, hg18_seq_base, str(record.seq[mut_pos-2:mut_pos+3]).upper(),line])

In [1]:
misaligned_lines[0:1]

NameError: name 'misaligned_lines' is not defined

In [68]:
len(misaligned_lines)

1489

# **lets try downloading hg19 seqeunce to compare to directly**

In [4]:
! mkdir ../../../data/skin/mutations/SomaMutDB/hg19_seq/

In [5]:
%%bash 

wget http://hgdownload.soe.ucsc.edu/goldenPath/hg18/chromosomes/chr3.fa.gz -P ../../../data/skin/mutations/SomaMutDB/hg19_seq/

--2023-05-29 09:39:52--  http://hgdownload.soe.ucsc.edu/goldenPath/hg18/chromosomes/chr3.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63962094 (61M) [application/x-gzip]
Saving to: ‘../../../data/skin/mutations/SomaMutDB/hg19_seq/chr3.fa.gz’

     0K .......... .......... .......... .......... ..........  0%  269K 3m52s
    50K .......... .......... .......... .......... ..........  0%  582K 2m50s
   100K .......... .......... .......... .......... ..........  0%  107M 1m53s
   150K .......... .......... .......... .......... ..........  0%  152M 85s
   200K .......... .......... .......... .......... ..........  0%  614K 88s
   250K .......... .......... .......... .......... ..........  0% 93.1M 73s
   300K .......... .......... .......... .......... ..........  0%  616K 77s
   350K .......... 

In [6]:
%%bash 
cd ../../../data/skin/mutations/SomaMutDB/
head processedMuts_hg_7.txt

chr1	107123428	107123429	AG	A
chr1	158074427	158074428	TG	T
chr2	6359430	6359431	ACT	A
chr2	50900803	50900804	TA	T
chr2	116275188	116275189	TC	T
chr2	171240283	171240284	G	A
chr2	216574848	216574849	TAAAGACCATTCTC	T
chr3	138724787	138724788	G	A
chr4	25430034	25430035	GT	G
chr4	36132113	36132114	TA	T


In [1]:
import pandas as pd 
import numpy as np
import gzip 
from Bio import SeqIO

In [8]:
%%bash 
cd ../../../data/skin/mutations/SomaMutDB/
grep "chr3" processedMuts_hg_7.txt > hg19_seq/skin_mutations_hg19_chr3.bed

In [9]:
muts_readlines = open("../../../data/skin/mutations/SomaMutDB/hg19_seq/skin_mutations_hg19_chr3.bed").readlines()

In [10]:
for line in muts_readlines: 
    assert line.split()[0] == "chr3"

In [11]:
muts_readlines[0:10]

['chr3\t138724787\t138724788\tG\tA\n',
 'chr3\t47513767\t47513768\tCA\tC\n',
 'chr3\t100918112\t100918113\tC\tCC\n',
 'chr3\t134911642\t134911643\tG\tA\n',
 'chr3\t134911643\t134911644\tG\tA\n',
 'chr3\t165034173\t165034174\tAT\tA\n',
 'chr3\t18309239\t18309240\tT\tA\n',
 'chr3\t42170471\t42170472\tGC\tG\n',
 'chr3\t14525813\t14525814\tGC\tG\n',
 'chr3\t122380502\t122380503\tGTACCTCCCAGTCCAGTCGAGCTGCTTCTATC\tG\n']

In [14]:
aligned_lines = []
misaligned_lines = []
with gzip.open("../../../data/skin/mutations/SomaMutDB/hg19_seq/chr3.fa.gz", "rt") as handle: #https://stackoverflow.com/questions/42757283/seqio-parse-on-a-fasta-gz
    for record in SeqIO.parse(handle,"fasta"): 
        for line in  muts_readlines: 
            chrom,mut_pos,mut_pos_plus1,mut_data_ref,mut_data_alt = line.split("\t")
            mut_pos=int(mut_pos)
            hg18_seq_base = record.seq[mut_pos].upper()
#             if hg18_seq_base != "N":
#                 assert mut_data_ref == hg18_seq_base
            if mut_data_ref != hg18_seq_base: 
                misaligned_lines.append([mut_pos, mut_data_ref, hg18_seq_base, str(record.seq[mut_pos-2:mut_pos+3]).upper(),line])
            else: 
                aligned_lines.append([mut_pos, mut_data_ref, hg18_seq_base, str(record.seq[mut_pos-2:mut_pos+3]).upper(),line])

In [15]:
len(muts_readlines)

181

In [16]:
len(aligned_lines)

34

In [18]:
len(misaligned_lines)/len(muts_readlines)

0.8121546961325967

### **ok even more basic: lets test a file idrectly before combining all of them together**

In [30]:
%%bash 
cd ../../../data/skin/mutations/SomaMutDB/
grep "^3"  hg19_7/DAG_H95.vcf > hg19_seq/DAG_H95_chr3_hg29_muts.vcf

In [32]:
%%bash 
cd ../../../data/skin/mutations/SomaMutDB/
head hg19_seq/DAG_H95_chr3_hg29_muts.vcf

3	4733007	.	C	T	.	.	.	.	.
3	4733008	.	C	T	.	.	.	.	.
3	8506356	.	A	AC	.	.	.	.	.
3	49848012	.	T	C	.	.	.	.	.
3	52588749	.	G	A	.	.	.	.	.
3	52588750	.	G	A	.	.	.	.	.
3	62739437	.	C	T	.	.	.	.	.
3	64133053	.	G	A	.	.	.	.	.
3	64133054	.	G	A	.	.	.	.	.
3	119256132	.	G	A	.	.	.	.	.


In [2]:
muts_readlines = open("../../../data/skin/mutations/SomaMutDB/hg19_seq/DAG_H95_chr3_hg29_muts.vcf").readlines()

In [3]:
for line in muts_readlines: 
    assert line.split()[0] == "3"

In [35]:
muts_readlines[0:10]

['3\t4733007\t.\tC\tT\t.\t.\t.\t.\t.\n',
 '3\t4733008\t.\tC\tT\t.\t.\t.\t.\t.\n',
 '3\t8506356\t.\tA\tAC\t.\t.\t.\t.\t.\n',
 '3\t49848012\t.\tT\tC\t.\t.\t.\t.\t.\n',
 '3\t52588749\t.\tG\tA\t.\t.\t.\t.\t.\n',
 '3\t52588750\t.\tG\tA\t.\t.\t.\t.\t.\n',
 '3\t62739437\t.\tC\tT\t.\t.\t.\t.\t.\n',
 '3\t64133053\t.\tG\tA\t.\t.\t.\t.\t.\n',
 '3\t64133054\t.\tG\tA\t.\t.\t.\t.\t.\n',
 '3\t119256132\t.\tG\tA\t.\t.\t.\t.\t.\n']

In [31]:
aligned_lines = []
misaligned_lines = []
weird_lines = []
with gzip.open("../../../data/skin/mutations/SomaMutDB/hg19_seq/chr3.fa.gz", "rt") as handle: #https://stackoverflow.com/questions/42757283/seqio-parse-on-a-fasta-gz
    for record in SeqIO.parse(handle,"fasta"): 
        for line in  muts_readlines: 
            chrom,mut_pos,mut_id,mut_data_ref,mut_data_alt = line.split("\t")[0:5]
            mut_pos=int(mut_pos)
            hg19_seq_base = record.seq[mut_pos].upper()
#             if hg18_seq_base != "N":
#                 assert mut_data_ref == hg18_seq_base
            if mut_data_alt == hg19_seq_base: 
                weird_lines.append([mut_pos, mut_data_ref, hg19_seq_base, str(record.seq[mut_pos-2:mut_pos+3]).upper(),line.split()])
            elif mut_data_ref != hg19_seq_base: 
                misaligned_lines.append([mut_pos, mut_data_ref, hg19_seq_base, str(record.seq[mut_pos-2:mut_pos+3]).upper(),line.split()])
            else:
                aligned_lines.append([mut_pos, mut_data_ref, hg19_seq_base, str(record.seq[mut_pos-2:mut_pos+3]).upper(),line.split()])\
                
#             if mut_pos == 4733007: 
#                 break 

In [32]:
len(weird_lines)

7

In [33]:
len(muts_readlines)

15

In [34]:
len(misaligned_lines)

5

In [35]:
len(aligned_lines)

3

### **ok lets go back and repeat for all combined to see how many times the mutant allele matches the refrence genome...**

In [36]:
muts_readlines = open("../../../data/skin/mutations/SomaMutDB/hg19_seq/skin_mutations_hg19_chr3.bed").readlines()

In [37]:
for line in muts_readlines: 
    assert line.split()[0] == "chr3"

In [ ]:
aligned_lines = []
misaligned_lines = []
weird_lines = []
with gzip.open("../../../data/skin/mutations/SomaMutDB/hg19_seq/chr3.fa.gz", "rt") as handle: #https://stackoverflow.com/questions/42757283/seqio-parse-on-a-fasta-gz
    for record in SeqIO.parse(handle,"fasta"): 
        for line in  muts_readlines: 
            chrom,mut_pos,mut_pos_plus1,mut_data_ref,mut_data_alt = line.split("\t")
            mut_pos=int(mut_pos)
            hg18_seq_base = record.seq[mut_pos].upper()
#             if hg18_seq_base != "N":
#                 assert mut_data_ref == hg18_seq_base
            if mut_data_alt == hg19_seq_base: 
                weird_lines.append([mut_pos, mut_data_ref, hg19_seq_base, str(record.seq[mut_pos-2:mut_pos+3]).upper(),line.split()])
            elif mut_data_ref != hg19_seq_base: 
                misaligned_lines.append([mut_pos, mut_data_ref, hg19_seq_base, str(record.seq[mut_pos-2:mut_pos+3]).upper(),line.split()])
            else: 
                aligned_lines.append([mut_pos, mut_data_ref, hg18_seq_base, str(record.seq[mut_pos-2:mut_pos+3]).upper(),line])

In [43]:
len(muts_readlines)

181

In [44]:
len(weird_lines)

0

In [45]:
len(misaligned_lines)

135

In [46]:
len(aligned_lines)

46

### **ok weird, now there are no lines that go into "weird" --> but the DAG_H95.vcf lines should have been included...***
- ok try again to copy all files into one, but include souce filename 

In [7]:
%%bash
#create a file with all the list names 
cd ../../../data/skin/mutations/SomaMutDB/
head list_7_muts.txt

allMuts_7.txt
hg19_7
hg19_7_filelist.txt
hg19_7.tar.gz
hg19_7.tar.gz.1
hg19_seq
liftover_unlifted_hg19_to_18.bed
not_working
processedMuts_hg_7.txt
skin_mutations_hg18.bed
skin_mutations_hg18.bed.gz.tbi
skin_mutations_hg18_chr1.bed
skin_mutations_hg18_chr2.bed
skin_mutations_hg18_chr4.bed
wget_7_output.txt


In [5]:
%%bash
#create a file with all the list names 
cd ../../../data/skin/mutations/SomaMutDB/
awk '{print $0}' allMuts_7.txt  | head 

1	107123428	.	AG	A	.	.	.	.	.
1	158074427	.	TG	T	.	.	.	.	.
2	6359430	.	ACT	A	.	.	.	.	.
2	50900803	.	TA	T	.	.	.	.	.
2	116275188	.	TC	T	.	.	.	.	.
2	171240283	.	G	A	.	.	.	.	.
2	216574848	.	TAAAGACCATTCTC	T	.	.	.	.	.
3	138724787	.	G	A	.	.	.	.	.
4	25430034	.	GT	G	.	.	.	.	.
4	36132113	.	TA	T	.	.	.	.	.


# **download hg38**

In [9]:
%%bash 
wget http://vijglab.einsteinmed.org/static/vcf/hg38_7.tar.gz --no-check-certificate

--2023-05-29 19:18:41--  http://vijglab.einsteinmed.org/static/vcf/hg38_7.tar.gz
Resolving vijglab.einsteinmed.org (vijglab.einsteinmed.org)... 103.41.68.205
Connecting to vijglab.einsteinmed.org (vijglab.einsteinmed.org)|103.41.68.205|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://vijglab.einsteinmed.org/static/vcf/hg38_7.tar.gz [following]
--2023-05-29 19:18:41--  https://vijglab.einsteinmed.org/static/vcf/hg38_7.tar.gz
Connecting to vijglab.einsteinmed.org (vijglab.einsteinmed.org)|103.41.68.205|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 26946 (26K) [application/x-tar]
Saving to: ‘hg38_7.tar.gz’

     0K .......... .......... ......                          100%  104M=0s

2023-05-29 19:18:42 (104 MB/s) - ‘hg38_7.tar.gz’ saved [26946/26946]



In [13]:
!mkdir hg38_7/

In [14]:
! tar -xzf  hg38_7.tar.gz -C hg38_7/

**ok so i did a few lines in the web app liftover and they are the same as the hg19, which gave an error so its not liftover**

# **try new daatset**